#Extract graphs for 2 time period

In [ ]:
import requests
import pandas as pd
from collections import OrderedDict
import numpy as np
from itertools import permutations 
import tqdm

In [ ]:
# cluster1 = ['http://dbpedia.org/resource/PHP']
# cluster2 = ['http://dbpedia.org/resource/JavaScript']

# cluster1 = ['http://dbpedia.org/resource/PHP', 'http://dbpedia.org/resource/ECMAScript']
# cluster2 = ['http://dbpedia.org/resource/JavaScript', 'http://dbpedia.org/resource/Dynamic_HTML', 'http://dbpedia.org/resource/PHP']

# cluster1 = ['http://dbpedia.org/resource/Fortran', 'http://dbpedia.org/resource/JavaScript', 'http://dbpedia.org/resource/PHP', 'http://dbpedia.org/resource/C++']
# cluster2 = ['http://dbpedia.org/resource/ECMAScript', 'http://dbpedia.org/resource/JavaScript', 'http://dbpedia.org/resource/Dynamic_HTML', 'http://dbpedia.org/resource/VBScript']

In [ ]:
# getPathsBetweenTwoNodes Function returns dataframe of All possible paths between two Nodes
def getPathsBetweenTwoNodes(A, B): 
  url = 'http://dbpedia.org/sparql/'
  query = """
  SELECT *
  WHERE
  {
    {
    <""" + A + """>  ?r1 <""" + B + """> .
    }
    UNION
    {
      <""" + A + """> ?r1 ?node1 .
      ?node1 ?r2 <""" + B + """> .
    }
    UNION
    {
      <""" + A + """> ?r1 ?node1 .
      ?node1 ?r2 ?node2 .
      ?node2 ?r3 <""" + B + """> .
    }
    # Add additional UNION for each length of path you want up to your upper bound
  }
  """
  print(query)
  r = requests.get(url, params = {'format': 'json', 'query': query})
  data = r.json()

  subgraph = []
  for item in data['results']['bindings']:
      subgraph.append(OrderedDict({
        'source_node': A, 
        'r1': item['r1']['value'],
        'node1': item['node1']['value']
            if 'node1' in item else "",
        'r2': item['r2']['value']
            if 'r2' in item else "",
        'node2': item['node2']['value']
            if 'node2' in item else "",
        'r3': item['r3']['value']
            if 'r3' in item else "",
        'target_node': B
        }))

  df = pd.DataFrame(subgraph)
  return df

In [ ]:
getPathsBetweenTwoNodes('http://dbpedia.org/resource/PHP', 'http://dbpedia.org/resource/JavaScript')


  SELECT *
  WHERE
  {
    {
    <http://dbpedia.org/resource/PHP>  ?r1 <http://dbpedia.org/resource/JavaScript> .
    }
    UNION
    {
      <http://dbpedia.org/resource/PHP> ?r1 ?node1 .
      ?node1 ?r2 <http://dbpedia.org/resource/JavaScript> .
    }
    UNION
    {
      <http://dbpedia.org/resource/PHP> ?r1 ?node1 .
      ?node1 ?r2 ?node2 .
      ?node2 ?r3 <http://dbpedia.org/resource/JavaScript> .
    }
    # Add additional UNION for each length of path you want up to your upper bound
  }
  


,source_node,r1,node1,r2,node2,r3,target_node
0,http://dbpedia.org/resource/PHP,http://dbpedia.org/ontology/wikiPageWikiLink,,,,,http://dbpedia.org/resource/JavaScript
1,http://dbpedia.org/resource/PHP,http://dbpedia.org/property/influencedBy,,,,,http://dbpedia.org/resource/JavaScript
2,http://dbpedia.org/resource/PHP,http://dbpedia.org/ontology/influencedBy,,,,,http://dbpedia.org/resource/JavaScript
3,http://dbpedia.org/resource/PHP,http://dbpedia.org/ontology/wikiPageWikiLink,http://dbpedia.org/resource/Constructor_(objec...,http://dbpedia.org/ontology/wikiPageWikiLink,,,http://dbpedia.org/resource/JavaScript
4,http://dbpedia.org/resource/PHP,http://dbpedia.org/ontology/wikiPageWikiLink,http://dbpedia.org/resource/Control_flow,http://dbpedia.org/ontology/wikiPageWikiLink,,,http://dbpedia.org/resource/JavaScript
...,...,...,...,...,...,...,...
8751,http://dbpedia.org/resource/PHP,http://dbpedia.org/ontology/wikiPageWikiLink,http://dbpedia.org/resource/Suhosin,http://dbpedia.org/ontology/programmingLanguage,http://dbpedia.org/resource/C_(programming_lan...,http://dbpedia.org/ontology/influenced,http://dbpedia.org/resource/JavaScript
8752,http://dbpedia.org/resource/PHP,http://dbpedia.org/ontology/wikiPageWikiLink,http://dbpedia.org/resource/Phalcon_(framework),http://dbpedia.org/ontology/programmingLanguage,http://dbpedia.org/resource/C_(programming_lan...,http://dbpedia.org/ontology/influenced,http://dbpedia.org/resource/JavaScript
8753,http://dbpedia.org/resource/PHP,http://dbpedia.org/ontology/wikiPageWikiLink,http://dbpedia.org/resource/PostgreSQL,http://dbpedia.org/ontology/programmingLanguage,http://dbpedia.org/resource/C_(programming_lan...,http://dbpedia.org/ontology/influenced,http://dbpedia.org/resource/JavaScript
8754,http://dbpedia.org/resource/PHP,http://dbpedia.org/ontology/wikiPageWikiLink,http://dbpedia.org/resource/HipHop_for_PHP,http://dbpedia.org/ontology/programmingLanguage,http://dbpedia.org/resource/C_(programming_lan...,http://dbpedia.org/ontology/influenced,http://dbpedia.org/resource/JavaScript


In [ ]:
def getFullGraph(cluster1, cluster2):
  clusterNodesList = cluster1 + cluster2
  clusterNodesList = list(dict.fromkeys(clusterNodesList))

  perm = permutations(clusterNodesList, 2) 
  frames = []

  for element in list(perm):
    df = getPathsBetweenTwoNodes(element[0], element[1])
    frames.append(df)

  result = pd.concat(frames)
  return result

In [ ]:
# c1 = ['http://dbpedia.org/resource/User_experience', 'http://dbpedia.org/resource/NoSQL', 'http://dbpedia.org/resource/User_interface', 'http://dbpedia.org/resource/Computer_hardware', 'http://dbpedia.org/resource/Relational_database', 'http://dbpedia.org/resource/Project_management', 'http://dbpedia.org/resource/SCADA']
# c2 = ['http://dbpedia.org/resource/Software_architecture', 'http://dbpedia.org/resource/Microservices', 'http://dbpedia.org/resource/Virtual_machine']
# x = getFullGraph(c1, c2)

#Get the set of all unique Nodes and Edge sets

In [ ]:
def getUniqueNodesFromGraph(nodes_filter_graph):
  # Get all Unique Nodes
  unique_nodes_set = list(nodes_filter_graph.source_node.unique()) + list(nodes_filter_graph.node1.unique()) + list(nodes_filter_graph.node2.unique()) + list(nodes_filter_graph.target_node.unique())
  unique_nodes_set = np.unique(unique_nodes_set)
  # storing unique nodes in dataframe
  unique_nodes_df = pd.DataFrame(unique_nodes_set.tolist(), columns=['Node'])
  unique_nodes_df["id"] = unique_nodes_df.index
  unique_nodes_df.head()
  return unique_nodes_df

def getUniqueRelationsFromGraph(graphFull):
  #Filtered table with only relations
  edge_type_filter = graphFull[['r1', 'r2', 'r3']]
  # Get set of edge types in the graph
  edge_type_set = list(edge_type_filter.r1.unique()) + list(edge_type_filter.r2.unique()) + list(edge_type_filter.r3.unique())
  edge_type_set = np.unique(edge_type_set)

  # storing unique edges in dataframe
  unique_edges_df = pd.DataFrame(edge_type_set.tolist(), columns=['Edge'])
  unique_edges_df["id"] = unique_edges_df.index
  unique_edges_df.head(18)
  return unique_edges_df

#Map node and edge to respective id

In [ ]:
def mapNodeWithId(node):
  if node != "":
    df = unique_nodes_df.query('Node == "' + node + '"', inplace = False)
    id = df["id"].tolist()
    return str(id[0])
  else:
    return ""

def mapPredicateWithId(predicate):
  if predicate != "":
    df = unique_edges_df.query('Edge == "' + predicate + '"', inplace = False)
    id = df["id"].tolist()
    return str(id[0])
  else:
    return ""

**Get the Edge set**

In [ ]:
pd.set_option('max_rows', 99999)
pd.set_option('max_colwidth', 400)
pd.describe_option('max_colwidth')

display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: 400]


In [ ]:
def getAllRelationSets(graphFull):
  # Subject -- Predicate -- object
  relation_set = []
  for index, row in graphFull.iterrows():
    # for r1
    if row["node1"] == '':
      relation_set.append([row["source_node"], row["r1"], row["target_node"]])
    else:
      relation_set.append([row["source_node"], row["r1"], row["node1"]])
    # for r2
    if row["node2"] == '' and row["node1"] != '':
      relation_set.append([row["node1"], row["r2"], row["target_node"]])
    else:
      relation_set.append([row["node1"], row["r2"], row["node2"]])
    # for r3
    if row["node2"] != '':
      relation_set.append([row["node2"], row["r3"], row["target_node"]])

  relation_Dataframe = pd.DataFrame(relation_set, columns= ['start', 'relation', 'end'])
  return relation_Dataframe

#Group Predicates, objects and object-predicate pair with their number of occurence

**Unique Relation Dataset**

In [ ]:
def getWeightCalcHelpers(relation_Dataframe):
  unique_relations_count_series = relation_Dataframe.groupby(['start', 'relation', 'end']).size()
  unique_relations_count_df = unique_relations_count_series.to_frame(name = 'size').reset_index()
  unique_relation_Dataframe = unique_relations_count_df.copy()
  unique_relation_Dataframe.drop(['size'], axis = 1)

  predicate_count_series = relation_Dataframe.groupby(['relation']).size()
  predicate_count_df = predicate_count_series.to_frame(name = 'size').reset_index()

  object_count_series = relation_Dataframe.groupby(['end']).size()
  object_count_df = object_count_series.to_frame(name = 'size').reset_index()

  pred_obj_series = relation_Dataframe.groupby(['relation', 'end']).size()
  pred_obj_count_df = pred_obj_series.to_frame(name = 'size').reset_index()

  return unique_relation_Dataframe, predicate_count_df, object_count_df, pred_obj_count_df

#Semantic relation weighting

**Calculate IC for predicate and object**

In [ ]:
def getICpredicate(predicate, predicate_count_df, all_pred_count):
  given_pred_count = list(predicate_count_df.query('relation == "' + predicate + '"', inplace = False)["size"])[0]
  prob_pred = given_pred_count/all_pred_count
  ic_pred = -np.log(prob_pred)
  return ic_pred

In [ ]:
def getICobj(obj, object_count_df, all_obj_count):
  given_obj_count = list(object_count_df.query('end == "' + obj + '"', inplace = False)["size"])[0]
  prob_obj = given_obj_count/all_obj_count
  ic_obj = -np.log(prob_obj)
  return ic_obj

In [ ]:
def getICobj_given_pred(obj, pred, predicate_count_df, pred_obj_count_df, total_relation):
  given_pred_count = list(predicate_count_df.query('relation == "' + pred + '"', inplace = False)["size"])[0]

  prob_pred = given_pred_count/total_relation

  obj_and_pred_count = list(pred_obj_count_df.query('end == "'+ obj + '" and relation == "' + pred + '"', inplace = False)["size"])[0]

  prob_obj_and_pred = obj_and_pred_count/total_relation

  prob_obj_given_pred = prob_obj_and_pred/prob_pred
  
  ic_obj_and_pred = -np.log(prob_obj_given_pred)
  return ic_obj_and_pred

In [ ]:
def getPMI(obj, pred, total_relation, predicate_count_df, pred_obj_count_df, object_count_df):
  given_pred_count = list(predicate_count_df.query('relation == "' + pred + '"', inplace = False)["size"])[0]

  prob_pred = given_pred_count/total_relation

  obj_and_pred_count = list(pred_obj_count_df.query('end == "'+ obj + '" and relation == "' + pred + '"', inplace = False)["size"])[0]

  prob_obj_and_pred = obj_and_pred_count/total_relation

  prob_obj_given_pred = prob_obj_and_pred/prob_pred

  given_obj_count = list(object_count_df.query('end == "' + obj + '"', inplace = False)["size"])[0]
  prob_obj = given_obj_count/total_relation

  pmi = np.log(prob_obj_given_pred/prob_obj)
  return pmi

**Joint IC, combIC and ic+pmi**

In [ ]:
def jointIC(pred, obj, predicate_count_df, pred_obj_count_df, all_pred_count, total_relation):
  ic_pred = getICpredicate(pred, predicate_count_df, all_pred_count)
  ic_obj_given_pred = getICobj_given_pred(obj, pred, predicate_count_df, pred_obj_count_df, total_relation)
  weight = ic_pred + ic_obj_given_pred
  return weight

In [ ]:
def combIC(pred, obj, predicate_count_df, object_count_df, all_pred_count, all_obj_count):
  ic_pred = getICpredicate(pred, predicate_count_df, all_pred_count)
  ic_obj = getICobj(obj, object_count_df, all_obj_count)
  weight = ic_pred + ic_obj
  return weight

In [ ]:
def ic_and_pmi(pred, obj, predicate_count_df, pred_obj_count_df, object_count_df, all_pred_count, total_relation):
  ic_pred = getICpredicate(pred, predicate_count_df, all_pred_count)
  pmi_pred_obj = getPMI(obj, pred, total_relation, predicate_count_df, pred_obj_count_df, object_count_df)
  weight = ic_pred + pmi_pred_obj
  return weight

#Assign Weights on relation Dataframe

**Joint IC, combIC, ic&pmi Scheme**

In [ ]:
def getWeightOfEdgeWithExclusivity(dataset, source, predicate, target):
  # calculate exclusivity
  outgoingPredicatesCount = dataset.query(
            'start == "'+ source + '" and relation == "' + predicate + '"', inplace = False).shape[0]
  incomingPredicatesCount = dataset.query(
            'end == "'+ target + '" and relation == "' + predicate + '"', inplace = False).shape[0]
  exclusivity = 1/(outgoingPredicatesCount + incomingPredicatesCount - 1)
  return exclusivity

In [ ]:
def assignWeightsToEdges(all_relation_Dataframe, unique_relation_Dataframe, predicate_count_df, object_count_df, pred_obj_count_df, all_pred_count, all_obj_count, total_relation):
  # Assign Weights to Edge set 
  edge_set_with_weight = []
  for index, row in unique_relation_Dataframe.iterrows():
    if row["start"] != "" and row["relation"] != "" and row["end"] != "":
      joint_ic_weight = jointIC(row["relation"], row["end"], predicate_count_df, pred_obj_count_df, all_pred_count, total_relation)
      comb_ic_weight = combIC(row["relation"], row["end"], predicate_count_df, object_count_df, all_pred_count, all_obj_count)
      ic_pmi_weight = ic_and_pmi(row["relation"], row["end"], predicate_count_df, pred_obj_count_df, object_count_df, all_pred_count, total_relation)
      exclusivity = getWeightOfEdgeWithExclusivity(all_relation_Dataframe, row["start"], row["relation"], row["end"])
      edge_set_with_weight.append([row["start"], row["relation"], row["end"], joint_ic_weight, comb_ic_weight, ic_pmi_weight, exclusivity])
      # edge_set_with_weight.append([row["start"], row["relation"], row["end"], joint_ic_weight])
  edge_set_weight_DF = pd.DataFrame(edge_set_with_weight, columns= ['Start', 'Relation', 'End', 'Joint_IC_Weight', 'Comb_IC_Weight', 'IC_PMI_weight', 'exclusivity'])
  # edge_set_weight_DF = pd.DataFrame(edge_set_with_weight, columns= ['Start', 'Relation', 'End', 'Joint_IC_Weight'])
  return edge_set_weight_DF

#Compute Cost of all the Edges
---
c(e) = wmax - w (e) ;


In [ ]:
def assignCostToEdges(edge_set_weight_DF):
  edge_set_ic_Cost = []
  max_joint_weight = edge_set_weight_DF['Joint_IC_Weight'].max()
  max_comb_weight = edge_set_weight_DF['Comb_IC_Weight'].max()
  max_ic_pmi_weight = edge_set_weight_DF['IC_PMI_weight'].max()

  for index, row in edge_set_weight_DF.iterrows():
    joint_cost = max_joint_weight - row["Joint_IC_Weight"] + 0.001
    comb_cost = max_comb_weight - row["Comb_IC_Weight"] + 0.001
    ic_pmi_cost = max_ic_pmi_weight - row["IC_PMI_weight"] + 0.001
    edge_set_ic_Cost.append([row["Start"], row["Relation"], row["End"], row["Joint_IC_Weight"], joint_cost, row["Comb_IC_Weight"], comb_cost, row["IC_PMI_weight"], ic_pmi_cost, row["exclusivity"]])
    # edge_set_ic_Cost.append([row["Start"], row["Relation"], row["End"], row["Joint_IC_Weight"], joint_cost])
  edge_set_weight_cost_DF = pd.DataFrame(edge_set_ic_Cost, columns= ['Start', 'Relation', 'End', 'Joint_IC_Weight', 'Joint_Cost', 'Comb_IC_Weight', 'Comb_cost', 'IC_PMI_weight', 'IC_PMI_Cost', 'exclusivity'])
  # edge_set_weight_cost_DF = pd.DataFrame(edge_set_ic_Cost, columns= ['Start', 'Relation', 'End', 'Joint_IC_Weight', 'Joint_Cost'])
  return edge_set_weight_cost_DF

#Assign total weight and total cost (with 3 schemes) of all paths in graphFull dataset

In [ ]:
def assignWeightAndCostToGraph(graphFull, edge_set_weight_cost_DF):
  graphWithPathWeight = []
  for index, row in graphFull.iterrows():
    joint_weight = 0
    comb_weight = 0
    ic_pmi_weight = 0
    if row["node1"] == '':   #only one relation between source and target
      df = edge_set_weight_cost_DF.query('Start == "'+ row["source_node"] + '" and Relation == "' + row["r1"] + '" and End == "' + row["target_node"] + '"', inplace = False)
      joint_weight = df["Joint_IC_Weight"].values[0]
      comb_weight = df["Comb_IC_Weight"].values[0]
      ic_pmi_weight = df["IC_PMI_weight"].values[0]
      exclusivity = df["exclusivity"].values[0]
      exclusivity_weight = 1/ (1/exclusivity)

      joint_cost = df["Joint_Cost"].values[0]
      comb_cost = df["Comb_cost"].values[0]
      ic_pmi_cost = df["IC_PMI_Cost"].values[0]
      graphWithPathWeight.append([row["source_node"], row["r1"], "", "", "", "", row["target_node"], joint_weight, comb_weight, ic_pmi_weight,exclusivity_weight, joint_cost, comb_cost, ic_pmi_cost])
      # graphWithPathWeight.append([row["source_node"], row["r1"], "", "", "", "", row["target_node"], joint_weight, joint_cost])
    elif row["node1"] != '' and row["node2"] == '':  # one intermediate node between source and target
      df1 = edge_set_weight_cost_DF.query('Start == "'+ row["source_node"] + '" and Relation == "' + row["r1"] + '" and End == "' + row["node1"] + '"', inplace = False)
      df2 = edge_set_weight_cost_DF.query('Start == "'+ row["node1"] + '" and Relation == "' + row["r2"] + '" and End == "' + row["target_node"] + '"', inplace = False)
      joint_weight = df1["Joint_IC_Weight"].values[0] + df2["Joint_IC_Weight"].values[0]
      comb_weight = df1["Comb_IC_Weight"].values[0] + df2["Comb_IC_Weight"].values[0]
      ic_pmi_weight = df1["IC_PMI_weight"].values[0] + df2["IC_PMI_weight"].values[0]
      exclusivity_weight = 1/ (1/df1["exclusivity"].values[0]) + (1/df2["exclusivity"].values[0])

      joint_cost = df1["Joint_Cost"].values[0] + df2["Joint_Cost"].values[0]
      comb_cost = df1["Comb_cost"].values[0] + df2["Comb_cost"].values[0]
      ic_pmi_cost = df1["IC_PMI_Cost"].values[0] + df2["IC_PMI_Cost"].values[0]
      graphWithPathWeight.append([row["source_node"], row["r1"], row["node1"], row["r2"], "", "", row["target_node"], joint_weight, comb_weight, ic_pmi_weight, exclusivity_weight, joint_cost, comb_cost, ic_pmi_cost])
      # graphWithPathWeight.append([row["source_node"], row["r1"], row["node1"], row["r2"], "", "", row["target_node"], joint_weight, joint_cost])
    else:
      df1 = edge_set_weight_cost_DF.query('Start == "'+ row["source_node"] + '" and Relation == "' + row["r1"] + '" and End == "' + row["node1"] + '"', inplace = False)
      df2 = edge_set_weight_cost_DF.query('Start == "'+ row["node1"] + '" and Relation == "' + row["r2"] + '" and End == "' + row["node2"] + '"', inplace = False)
      df3 = edge_set_weight_cost_DF.query('Start == "'+ row["node2"] + '" and Relation == "' + row["r3"] + '" and End == "' + row["target_node"] + '"', inplace = False)
      joint_weight = df1["Joint_IC_Weight"].values[0] + df2["Joint_IC_Weight"].values[0] + df3["Joint_IC_Weight"].values[0]
      comb_weight = df1["Comb_IC_Weight"].values[0] + df2["Comb_IC_Weight"].values[0] + df3["Comb_IC_Weight"].values[0]
      ic_pmi_weight = df1["IC_PMI_weight"].values[0] + df2["IC_PMI_weight"].values[0] + df3["IC_PMI_weight"].values[0]
      exclusivity_weight = 1/ (1/df1["exclusivity"].values[0]) + (1/df2["exclusivity"].values[0]) + (1/df3["exclusivity"].values[0])

      joint_cost = df1["Joint_Cost"].values[0] + df2["Joint_Cost"].values[0] +  + df3["Joint_Cost"].values[0]
      comb_cost = df1["Comb_cost"].values[0] + df2["Comb_cost"].values[0] + df3["Comb_cost"].values[0]
      ic_pmi_cost = df1["IC_PMI_Cost"].values[0] + df2["IC_PMI_Cost"].values[0] + df3["IC_PMI_Cost"].values[0]
      graphWithPathWeight.append([row["source_node"], row["r1"], row["node1"], row["r2"], row["node2"], row["r3"], row["target_node"], joint_weight, comb_weight, ic_pmi_weight, exclusivity_weight, joint_cost, comb_cost, ic_pmi_cost])
      # graphWithPathWeight.append([row["source_node"], row["r1"], row["node1"], row["r2"], row["node2"], row["r3"], row["target_node"], joint_weight, joint_cost])

  graphFullWith_ic_Weights_Cost = pd.DataFrame(graphWithPathWeight, columns= ['source_node', 'r1', 'node1', 'r2', 'node2', 'r3', 'target_node', 'Joint_Weight', 'Comb_Weight', 'IC_PMI_Weight', 'exclusivity_weight', 'Joint_Cost', 'Comb_Cost', 'IC_PMI_Cost'])
  # graphFullWith_ic_Weights_Cost = pd.DataFrame(graphWithPathWeight, columns= ['source_node', 'r1', 'node1', 'r2', 'node2', 'r3', 'target_node', 'Joint_Weight', 'Joint_Cost'])

  return graphFullWith_ic_Weights_Cost

**Normalize all the cost**

In [ ]:
def getGraphWithNormalizedCost(graphFullWith_ic_Weights_Cost):
  max_joint_cost = graphFullWith_ic_Weights_Cost['Joint_Cost'].max()
  max_comb_cost = graphFullWith_ic_Weights_Cost['Comb_Cost'].max()
  max_ic_pmi_cost = graphFullWith_ic_Weights_Cost['IC_PMI_Cost'].max()

  graphWithNormalizedCost = []
  for index, row in graphFullWith_ic_Weights_Cost.iterrows():
    normalized_joint = row["Joint_Cost"] / max_joint_cost
    normalized_comb = row["Comb_Cost"] / max_comb_cost
    normalized_ic_pmi = row["IC_PMI_Cost"] / max_ic_pmi_cost
    graphWithNormalizedCost.append([row["source_node"], row["r1"], row["node1"], row["r2"], row["node2"], row["r3"], row["target_node"], row["Joint_Weight"], normalized_joint, row["Comb_Weight"], normalized_comb, row["IC_PMI_Weight"], normalized_ic_pmi, row["exclusivity_weight"]])
    # graphWithNormalizedCost.append([row["source_node"], row["r1"], row["node1"], row["r2"], row["node2"], row["r3"], row["target_node"], row["Joint_Weight"], normalized_joint])

  graphWithNormalizedCost_DF = pd.DataFrame(graphWithNormalizedCost, columns= ['source_node', 'r1', 'node1', 'r2', 'node2', 'r3', 'target_node', 'Joint_Weight', 'Joint_Cost', 'Comb_Weight', 'Comb_cost', 'IC_PMI_Weight', 'IC_PMI_Cost', 'exclusivity_weight'])
  # graphWithNormalizedCost_DF = pd.DataFrame(graphWithNormalizedCost, columns= ['source_node', 'r1', 'node1', 'r2', 'node2', 'r3', 'target_node', 'Joint_Weight', 'Joint_Cost'])
  return graphWithNormalizedCost_DF

**Get Minimum cost between a pair of Nodes**

In [ ]:
def getMinimumCost(df, source, target, weight_scheme):
  filtered_df = df.query('source_node == "'+ source + '" and target_node == "' + target + '"', inplace = False)
  
  if filtered_df.empty:
    cols = ['source_node', 'r1', 'node1', 'r2', 'node2', 'r3', 'target_node', 'Joint_Weight', 'Joint_Cost', 'Comb_Weight', 'Comb_cost', 'IC_PMI_Weight', 'IC_PMI_Cost', 'exclusivity_weight']
    # cols = ['source_node', 'r1', 'node1', 'r2', 'node2', 'r3', 'target_node', 'Joint_Weight', 'Joint_Cost']
    temp_df = pd.concat([pd.DataFrame({k: [] for k in cols}), None, None])
    return temp_df
  
  if weight_scheme == 'comb_ic':
    return filtered_df[filtered_df.Comb_cost == filtered_df.Comb_cost.min()]
  elif weight_scheme == 'ic_pmi':
    return filtered_df[filtered_df.IC_PMI_Cost == filtered_df.IC_PMI_Cost.min()]
  elif weight_scheme == 'joint_ic':
    return filtered_df[filtered_df.Joint_Cost == filtered_df.Joint_Cost.min()]
  else:
    return filtered_df[filtered_df.exclusivity_weight == filtered_df.exclusivity_weight.max()]

In [ ]:
def getCheapestPath(df, source, target, weight_scheme):
  df1 = getMinimumCost(df, source, target, weight_scheme)
  df2 = getMinimumCost(df, target, source, weight_scheme)

  if weight_scheme == 'comb_ic':
    cost_name = 'Comb_cost'
  elif weight_scheme == 'ic_pmi':
    cost_name = 'IC_PMI_Cost'
  elif weight_scheme == 'joint_ic':
    cost_name = 'Joint_Cost'
  else:
    cost_name = 'exclusivity_weight'

  if not df1.empty:
    c1 = df1[cost_name].values[0]
  else:
    c1 = 1

  if not df2.empty:
    c2 = df2[cost_name].values[0]
  else:
    c2 = 1

  if cost_name == 'exclusivity_weight':
    if c1 > c2:
      return c1
    else:
      return c2
  else:
    if c1 < c2:
      return c1
    else:
      return c2

#Mapping Skills to Dbpedia links

In [ ]:
def annotate_with_Dbpedia_spotlight(text, confidence):
  # text preprocessing
  text = text.replace("_", " ").replace("-", " ")
  URL = "https://api.dbpedia-spotlight.org/en/annotate?text=" + text + "&confidence=" + str(confidence) + ""
  HEADERS = {'Accept': 'application/json'}
  response = requests.get(URL, headers=HEADERS)
  if response.status_code != 200:
    return 0
  
  json_obj = response.json()
  if "Resources" in json_obj:
    return json_obj["Resources"][0]['@URI']
  else:
    return 0

# annotate_with_Dbpedia_spotlight('Machine learning', 0.5)

In [ ]:
# skill_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Knowledge graph files/skill_name_list.csv")

**Map Skill name to Link**

In [ ]:
def map_skills(skills_to_be_mapped):
# skills_to_be_mapped = ['mysql','oracle','postgresql','database']
  cluster = []
  for skill in skills_to_be_mapped:
    link = annotate_with_Dbpedia_spotlight(skill, 0.5)
    if link == 0:
      confidence = 0.4
      while confidence >= 0:
        link = annotate_with_Dbpedia_spotlight(skill, confidence)
        confidence = confidence - 0.1
    if link == 0:
      link_list = list(skill_df.query('name == "'+ skill.lower() + '"', inplace = False)["link"])
      if link_list:
        link = link_list[0]
      else:
        link = 0
    cluster.append(link)
    print(link)
  return cluster

 
  link = annotate_with_Dbpedia_spotlight(skill, 0.5)
  if link == 0:
    confidence = 0.4
    while confidence >= 0:
      link = annotate_with_Dbpedia_spotlight(skill, confidence)
      confidence = confidence - 0.1
  if link == 0:
    link_list = list(skill_df.query('name == "'+ skill.lower() + '"', inplace = False)["link"])
    if link_list:
      link = link_list[0]
    else:
      link = 0
  return link

**Dbpedia link repository of all available skills in our Networks**

In [ ]:
# cluster_detail_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Particle Density Algo/results/mapping-results/clusters_details.csv')
# cluster_detail_df = cluster_detail_df[["Cluster"]]
# all_skills_list = []
# for index, rows in cluster_detail_df.iterrows():
#   cluster = rows["Cluster"].rstrip('#')
#   c_arr = cluster.split('#')
#   all_skills_list = all_skills_list + c_arr

# all_skills_list = list(set(all_skills_list))

In [ ]:
# skill_link_map_list = []
# for skill in all_skills_list:
#   link = map_single_skill(skill)
#   skill_link_map_list.append([skill, link])

# skill_link_map_df = pd.DataFrame(skill_link_map_list, columns=['skill', 'link'])


In [ ]:
# skill_link_map_df.to_csv('/content/drive/My Drive/Colab Notebooks/Knowledge graph files/skill_links_map.csv', index=False)

# Build Graph with Nodes

- And Assign weights and Cost to connecting Edges

In [ ]:
def buildGraphWithCost(cluster1, cluster2):
  print('Runing Sparql')
  graphFull = getFullGraph(cluster1, cluster2)
  if len(graphFull) == 0:
    return 0
  # Filtered table with only nodes
  nodes_filter_graph = graphFull[['source_node', 'node1', 'node2', 'target_node']]

  unique_nodes_df = getUniqueNodesFromGraph(nodes_filter_graph)
  unique_edges_df = getUniqueRelationsFromGraph(graphFull)

  relation_Dataframe = getAllRelationSets(graphFull)

  all_pred_count = relation_Dataframe.query('relation != ""', inplace = False).shape[0]
  all_obj_count = relation_Dataframe.query('end != ""', inplace = False).shape[0]
  total_relation = relation_Dataframe.query('relation != ""', inplace = False).shape[0]

  unique_relation_Dataframe, predicate_count_df, object_count_df, pred_obj_count_df = getWeightCalcHelpers(relation_Dataframe)

  # Assign Weights to Edge Sets
  print('Assigning Weights')
  edge_set_weight_DF = assignWeightsToEdges(relation_Dataframe, unique_relation_Dataframe, predicate_count_df, object_count_df, pred_obj_count_df, all_pred_count, all_obj_count, total_relation)

  # Compute Cost for Edge sets
  edge_set_weight_cost_DF = assignCostToEdges(edge_set_weight_DF)

  # Compute total weight and cost for all connecting paths between Nodes in the graph
  print('Assigning Cost to Graph')
  graphFullWith_ic_Weights_Cost = assignWeightAndCostToGraph(graphFull, edge_set_weight_cost_DF)
  # Normalized Cost
  graphWithNormalizedCost_DF = getGraphWithNormalizedCost(graphFullWith_ic_Weights_Cost)

  return graphWithNormalizedCost_DF

In [ ]:
# cluster1 = ['http://dbpedia.org/resource/Apache_Lucene', 'http://dbpedia.org/resource/Apache_Mahout', 'http://dbpedia.org/resource/Elasticsearch']
# cluster2 = ['http://dbpedia.org/resource/Thread_(computing)', 'http://dbpedia.org/resource/AngularJS', 'http://dbpedia.org/resource/Computer_hardware', 'http://dbpedia.org/resource/Nagios', 'http://dbpedia.org/resource/ASP.NET', 'http://dbpedia.org/resource/Surveillance', 'http://dbpedia.org/resource/IBM_WebSphere', 'http://dbpedia.org/resource/JQuery', 'http://dbpedia.org/resource/Model–view–controller', 'http://dbpedia.org/resource/Groovy_(programming_language)', 'http://dbpedia.org/resource/LabVIEW', 'http://dbpedia.org/resource/Scripting_language']

# normalize_df = buildGraphWithCost(cluster1, cluster2)
# normalize_df

# Prepare Edit Distance Matrix


Matrix:

```
#             Ecma  Javascript HTML VB
# Fortran    [ 0     x         y     z ]
# Javascript [ a     b         c     z ]
# PHP        [ a     b         c     z ]
# C++        [ a     b         c     z ]
```





**IC+PMI edit Distance Matrix**

In [ ]:
from scipy.optimize import linear_sum_assignment

**Prepare Matrix**

In [ ]:
def prepare_edit_cost_matrix(cluster1, cluster2, weight_scheme, graphWithNormalizedCost_DF):
  list_outer = []
  for item in cluster1:
    list_inner = []
    for element in cluster2:
      if item != element:
        total_cost = getCheapestPath(graphWithNormalizedCost_DF, element, item, weight_scheme)
      else:
        total_cost = 0
      list_inner.append(total_cost)
    list_outer.append(list_inner)
  final_matrix = np.array(list_outer, np.float64)
  return final_matrix

# Compare two time period Graphs

In [ ]:
def getPeriodsClusters(period_a, period_b):
  period_a_links = []
  period_b_links = []
  skill_link_map_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Knowledge graph files/skill_links_map.csv')
  cluster_detail_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Particle Density Algo/results/mapping-results/clusters_details.csv')

  period_a_clusters = cluster_detail_df.query('TimeLabel == "' + period_a + '"' , inplace=False)["Cluster"]
  for cluster in period_a_clusters:
    cluster = cluster.rstrip('#')
    items_arr = cluster.split('#')
    link_list = []
    for item in items_arr:
      link = skill_link_map_df.query('skill == "' + item + '"', inplace=False)["link"].tolist()[0]
      link_list.append(link)
    period_a_links.append(link_list)

  period_b_clusters = cluster_detail_df.query('TimeLabel == "' + period_b + '"' , inplace=False)["Cluster"]
  for cluster in period_b_clusters:
    cluster = cluster.rstrip('#')
    items_arr = cluster.split('#')
    link_list = []
    for item in items_arr:
      link = skill_link_map_df.query('skill == "' + item + '"', inplace=False)["link"].tolist()[0]
      link_list.append(link)
    period_b_links.append(link_list)
  return period_a_links, period_b_links

In [ ]:
period_a = "JulSep2017"
period_b = "OctDec2017"
period_a_links, period_b_links = getPeriodsClusters(period_a, period_b)

In [ ]:
period_a_links = [['http://dbpedia.org/resource/XML',
  'http://dbpedia.org/resource/Semantic_Web',
  'http://dbpedia.org/resource/HTTPS',
  'http://dbpedia.org/resource/SQL'],
 ['http://dbpedia.org/resource/Ansible',
  'http://dbpedia.org/resource/Virtualization',
  'http://dbpedia.org/resource/Linked_data',
  'http://dbpedia.org/resource/Elasticsearch',
  'http://dbpedia.org/resource/VMware',
  'http://dbpedia.org/resource/Relational_database',
  'http://dbpedia.org/resource/IBM_WebSphere',
  'http://dbpedia.org/resource/Data_modeling',
  'http://dbpedia.org/resource/Apache_Lucene',
  'http://dbpedia.org/resource/Microsoft_Visual_Studio',
  'http://dbpedia.org/resource/JSON',
  'http://dbpedia.org/resource/Redis',
  'http://dbpedia.org/resource/TypeScript',
  'http://dbpedia.org/resource/MySQL',
  'http://dbpedia.org/resource/PostgreSQL',
  'http://dbpedia.org/resource/The_Open_Group_Architecture_Framework',
  'http://dbpedia.org/resource/Version_control',
  'http://dbpedia.org/resource/Apache_Mahout',
  'http://dbpedia.org/resource/Jira',
  'http://dbpedia.org/resource/Node.js',
  'http://dbpedia.org/resource/JBoss_Enterprise_SOA_Platform',
  'http://dbpedia.org/resource/Database',
  'http://dbpedia.org/resource/Unit_testing',
  'http://dbpedia.org/resource/MongoDB',
  'http://dbpedia.org/resource/Perl',
  'http://dbpedia.org/resource/Oracle',
  'http://dbpedia.org/resource/Multicast',
  'http://dbpedia.org/resource/NoSQL',
  'http://dbpedia.org/resource/Gradle',
  'http://dbpedia.org/resource/Git'],
 ['http://dbpedia.org/resource/CUDA',
  'http://dbpedia.org/resource/Computer_hardware',
  'http://dbpedia.org/resource/OpenCL'],
 ['http://dbpedia.org/resource/Zend_Framework',
  'http://dbpedia.org/resource/PHP',
  'http://dbpedia.org/resource/Drupal',
  'http://dbpedia.org/resource/Laravel',
  'http://dbpedia.org/resource/Symfony'],
 ['http://dbpedia.org/resource/HTML',
  'http://dbpedia.org/resource/JavaScript',
  'http://dbpedia.org/resource/HTML5',
  'http://dbpedia.org/resource/AngularJS'],
 ['http://dbpedia.org/resource/Nagios',
  'http://dbpedia.org/resource/Surveillance',
  'http://dbpedia.org/resource/Linux_kernel',
  'http://dbpedia.org/resource/Nginx',
  'http://dbpedia.org/resource/LXC'],
 ['http://dbpedia.org/resource/Software_architecture',
  'http://dbpedia.org/resource/Microservices',
  'http://dbpedia.org/resource/Virtual_machine'],
 ['http://dbpedia.org/resource/Linux',
  'http://dbpedia.org/resource/Debian',
  'http://dbpedia.org/resource/Unix']]

In [ ]:
# period_b_links = 

In [ ]:
for cluster_a in period_a_links:
  joint_ic_cost_list = []
  ic_pmi_cost_list = []
  comb_ic_cost_list = []
  exclusivity_cost_list = []
  common_item_list = []
  for cluster_b in period_b_links:
    print("Processing")
    print(cluster_a)
    print(cluster_b)
    print('-------------')
    # common_items = list(set(cluster_a).intersection(cluster_b))
    # new_a = [elem for elem in cluster_a if elem not in common_items]
    # new_b = [elem for elem in cluster_b if elem not in common_items]
    new_a = cluster_a
    new_b = cluster_b

    # check if new_a is empty or new_b is empty
    if not new_a or not new_b:
      continue;

    normalized_cost_df = buildGraphWithCost(new_a, new_b)
    if type(normalized_cost_df) is not pd.DataFrame:
      print('hello')
      joint_ic_cost_list.append(100)
      ic_pmi_cost_list.append(100)
      comb_ic_cost_list.append(100)
      exclusivity_cost_list.append(0)
    else:
      joint_ic_matrix = prepare_edit_cost_matrix(new_a, new_b, 'joint_ic', normalized_cost_df)
      row_ind, col_ind = linear_sum_assignment(joint_ic_matrix)
      joint_ic_total_cost = joint_ic_matrix[row_ind, col_ind].sum()

      ic_pmi_matrix = prepare_edit_cost_matrix(new_a, new_b, 'ic_pmi', normalized_cost_df)
      row_ind, col_ind = linear_sum_assignment(ic_pmi_matrix)
      ic_pmi_total_cost = ic_pmi_matrix[row_ind, col_ind].sum()

      comb_ic_matrix = prepare_edit_cost_matrix(new_a, new_b, 'comb_ic', normalized_cost_df)
      row_ind, col_ind = linear_sum_assignment(comb_ic_matrix)
      comb_ic_total_cost = comb_ic_matrix[row_ind, col_ind].sum()

      exclusivity_matrix = prepare_edit_cost_matrix(new_a, new_b, 'exclusivity', normalized_cost_df)
      row_ind, col_ind = linear_sum_assignment(exclusivity_matrix)
      exclusivity_total_cost = exclusivity_matrix[row_ind, col_ind].sum()

      joint_ic_cost_list.append(joint_ic_total_cost)
      ic_pmi_cost_list.append(ic_pmi_total_cost)
      comb_ic_cost_list.append(comb_ic_total_cost)
      exclusivity_cost_list.append(exclusivity_total_cost)
    
  print("=============")
  print("Cluster: ", cluster_a, " with all:")
  print("=============")
  print("JointIC", joint_ic_cost_list)
  print("IC PMI", ic_pmi_cost_list)
  print("CombIC", comb_ic_cost_list)
  print("Exclusivity", exclusivity_cost_list)

Processing
['http://dbpedia.org/resource/XML', 'http://dbpedia.org/resource/Semantic_Web', 'http://dbpedia.org/resource/HTTPS', 'http://dbpedia.org/resource/SQL']
['http://dbpedia.org/resource/XML', 'http://dbpedia.org/resource/HTTPS', 'http://dbpedia.org/resource/Relational_database', 'http://dbpedia.org/resource/SQL', 'http://dbpedia.org/resource/JSON', 'http://dbpedia.org/resource/Redis', 'http://dbpedia.org/resource/MySQL', 'http://dbpedia.org/resource/PostgreSQL', 'http://dbpedia.org/resource/MongoDB', 'http://dbpedia.org/resource/NoSQL']
-------------
Runing Sparql
Assigning Weights
Assigning Cost to Graph
Processing
['http://dbpedia.org/resource/XML', 'http://dbpedia.org/resource/Semantic_Web', 'http://dbpedia.org/resource/HTTPS', 'http://dbpedia.org/resource/SQL']
['http://dbpedia.org/resource/Ansible', 'http://dbpedia.org/resource/Continuous_delivery', 'http://dbpedia.org/resource/Microservices', 'http://dbpedia.org/resource/Debian', 'http://dbpedia.org/resource/RabbitMQ']
---

In [ ]:
# Play an audio beep. Any audio URL will do.
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

# get exclusivity of a relation

In [ ]:
# def getWeightOfEdgeWithExclusivity(dataset, source, predicate, target):
#   # calculate exclusivity
#   outgoingPredicatesCount = dataset.query(
#             'start == "'+ source + '" and relation == "' + predicate + '"', inplace = False).shape[0]
#   incomingPredicatesCount = dataset.query(
#             'end == "'+ target + '" and relation == "' + predicate + '"', inplace = False).shape[0]
#   exclusivity = 1/(outgoingPredicatesCount + incomingPredicatesCount - 1)
#   return exclusivity

In [ ]:
# d = getWeightOfEdge(relation_Dataframe, 'http://dbpedia.org/resource/JavaScript', 'http://dbpedia.org/ontology/wikiPageWikiLink', 'http://dbpedia.org/resource/Google_Chrome')
# d

In [ ]:
#  # Assign Exclusivity to Edge set 
#  edge_set_with_exclusivity = []
#  for index, row in relation_Dataframe.iterrows():
#    if row["start"] != "" and row["relation"] != "" and row["end"] != "":
#      e = getWeightOfEdge(relation_Dataframe, row["start"], row["relation"], row["end"])
#      edge_set_with_exclusivity.append([row["start"], row["relation"], row["end"], e])
#  edge_set_with_exclusivity_DF = pd.DataFrame(edge_set_with_exclusivity, columns= ['Start', 'Relation', 'End', 'Exclusivity']) 
#  edge_set_with_exclusivity_DF.head(5)

In [ ]:
# edge_set_with_exclusivity_DF_filtered = edge_set_with_exclusivity_DF.copy()
# edge_set_with_exclusivity_DF_filtered.shape


In [ ]:
# edge_set_with_exclusivity_DF_filtered = edge_set_with_exclusivity_DF_filtered.drop_duplicates()
# edge_set_with_exclusivity_DF_filtered.head()
# edge_set_with_exclusivity_DF_filtered.shape